
# Combined Chain (Clean) — Prompt → LLM → Parser → Conditional → Parallel

## Goal
- Input: payment dict
- Generate disclaimer (English & French)
- If status == "completed":
  - Send Email & SMS in parallel
- Single composed chain with correct data flow


In [24]:

# Input
payment = {
    "amount": 1000,
    "payer_name": "Subbu",
    "payment_method": "Credit Card",
    "status": "completed"
}
print(payment)


{'amount': 1000, 'payer_name': 'Subbu', 'payment_method': 'Credit Card', 'status': 'completed'}



## Flow

Input
  ↓
Wrap for Prompt
  ↓
Prompt → LLM → Parser (Disclaimer)
  ↓
Merge (keep payment + disclaimer)
  ↓
Conditional Branch
  ↓
Parallel (Email + SMS) OR Default
  ↓
Final Output


In [25]:

# Schema
from pydantic import BaseModel

class Disclaimer(BaseModel):
    english: str
    french: str


In [26]:

# Parser
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Disclaimer)


In [27]:

# Prompt
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Generate payment disclaimer in English and French"),
    ("human", "Payment details: {payment}\n{format_instructions}")
])

prompt = prompt.partial(format_instructions=parser.get_format_instructions())


In [28]:

# LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    convert_system_message_to_human=True
)


In [29]:

# Base Chain (wrap input for prompt)
from langchain_core.runnables import RunnableLambda

payment_input = RunnableLambda(lambda input_payment: {"payment": input_payment})

base_chain = (
    payment_input
    | prompt
    | llm
    | parser
)


In [30]:

# Parallel Tasks
from langchain_core.runnables import RunnableParallel

def send_email(x):
    return f"Email sent to {x['payment']['payer_name']} for {x['payment']['amount']}"

def send_sms(x):
    return f"SMS sent to {x['payment']['payer_name']} for {x['payment']['amount']}"

parallel_chain = RunnableParallel({
    "email": RunnableLambda(send_email),
    "sms": RunnableLambda(send_sms)
})


In [31]:

# Conditional Branch
from langchain_core.runnables import RunnableBranch

def is_completed(x):
    return x["payment"]["status"] == "completed"

default_chain = RunnableLambda(lambda x: {"message": "No notification needed"})

branch_chain = RunnableBranch(
    (is_completed, parallel_chain),
    default_chain
)


In [32]:

# Merge Step (keep disclaimer + notifications)
def merge_result(x):
    return {
        "payment": x["payment"],
        "disclaimer": x["disclaimer"],
        "notifications": x.get("notifications", {})
    }

merge_chain = RunnableLambda(merge_result)


In [33]:

# Full Chain
from langchain_core.runnables import RunnablePassthrough

full_chain = (
    {
        "payment": RunnablePassthrough(),
        "disclaimer": base_chain
    }
    | RunnableLambda(lambda x: {
        **x,
        "notifications": branch_chain.invoke(x)
    })
    | merge_chain
)


In [34]:

# Execute
result = full_chain.invoke(payment)

print(result)
print("\nEnglish:", result["disclaimer"].english)
print("\nFrench:", result["disclaimer"].french)
print("\nNotifications:", result["notifications"])


{'payment': {'amount': 1000, 'payer_name': 'Subbu', 'payment_method': 'Credit Card', 'status': 'completed'}, 'disclaimer': Disclaimer(english='This is to confirm that a payment of 1000 has been successfully processed for Subbu using a Credit Card. Status: Completed. Please retain this receipt for your records as proof of transaction.', french="Ceci est pour confirmer qu'un paiement de 1000 a été traité avec succès pour Subbu via carte de crédit. État : Terminé. Veuillez conserver ce reçu pour vos dossiers comme preuve de transaction."), 'notifications': {'email': 'Email sent to Subbu for 1000', 'sms': 'SMS sent to Subbu for 1000'}}

English: This is to confirm that a payment of 1000 has been successfully processed for Subbu using a Credit Card. Status: Completed. Please retain this receipt for your records as proof of transaction.

French: Ceci est pour confirmer qu'un paiement de 1000 a été traité avec succès pour Subbu via carte de crédit. État : Terminé. Veuillez conserver ce reçu p